<img src="logoucm.png" style="height: 250px">
<center style="color:#888"><br/></center>




<font size="6"><center>***Metodos numéricos en finanzas***</center></font>

<font size="5"><center><span style="color:blue">***Nombre y Apellidos: ______________________________***</span></center></font>

## Importamos las librerías necesarias

In [6]:
import numpy as np
import pandas as pd
from math import sin, cos, sqrt, log, exp, pi
import matplotlib.pyplot as plt
import scipy as sp
from scipy.stats import t

In [7]:
M=2**(31)-1
A=16807
B=0

In [8]:
def imprimir_tabla_estadisticas(df):
    print("\n" + "="*75)
    print("      RESULTADOS MONTE CARLO - OPCION CALL EUROPEA VAINILLA")
    print("="*75)
    print(f"{'Plazos':<10} | {'Valor estimado':>15} | {'Varianza':>15} | {'Precisión (95%)':>18}")
    print("-"*75)

    for _, row in df.iterrows():
        print(f"{row['Plazo']:<10} | "
              f"{row['Valor estimado']:>15.6f} | "
              f"{row['Varianza']:>15.6f} | "
              f"{row['Precisión (95%)']:>18.6f}")

    print("="*75 + "\n")


def calcular_estadisticas_montecarlo(valores):
    n_muestras = len(valores)
    media = np.mean(valores)
    varianza = np.var(valores, ddof=1)
    desviacion_std = np.std(valores, ddof=1)
    t_critico = t.ppf(0.975, df=n_muestras - 1)

    precision = t_critico * (desviacion_std / sqrt(n_muestras))

    return media, varianza, precision


def generador_congruencial(semilla, m, a, b):
    x = semilla
    inv_m = 1 / m
    while True:
        x = (a * x + b) % m
        yield x * inv_m
def box_muller(N,ant=False):
    g_2 = generador_congruencial(123456789, M, A, B)
    n = N * 2
    U = [1 - next(g_2) if ant else next(g_2) for _ in range(n)]
    u1 = np.array(U[0::2])
    u2 = np.array(U[1::2])

    z0 = np.sqrt(-2.0 * np.log(u1)) * np.cos(2.0 * np.pi * u2)
    z1 = np.sqrt(-2.0 * np.log(u1)) * np.sin(2.0 * np.pi * u2)
    return z0, z1


def marsaglia_method(N,ant=False):
    g_3 = generador_congruencial(123456789, M, A, B)
    X = []
    Y = []
    n = 0
    while n<N:
        U_1 = 1-next(g_3) if ant else next(g_3)
        U_2 = 1-next(g_3) if ant else next(g_3)
        v_1 = 2 * U_1 - 1
        v_2 = 2 * U_2 - 1
        R_2 = v_1**2 + v_2**2
        if R_2 >= 1:
            continue

        y = np.sqrt(-2 * np.log(R_2) / R_2)
        X.append(v_1 * y)
        Y.append(v_2 * y)
        n += 1

    return np.array(X), np.array(Y)

def brownian_motion(T, N,X,Y):
    dt = T / N
    X = [0]+X.tolist()
    Y = [0]+Y.tolist()
    X = np.cumsum(np.sqrt(dt) * np.array(X))
    Y = np.cumsum(np.sqrt(dt) * np.array(Y))
    return X, Y


def Gamma(p,a,N):
    g_3 = generador_congruencial(123456789, M, A, B)
    X = []
    n=0
    while n < N:
        u = np.zeros(p)
        suma = 0
        for i in range(p):
            v = next(g_3)
            u[i] = log(v)
    
        x = -sum(u)/a
        X.append(x)
        n+=1
    return np.array(X)

def chi_cuadrado(n,N):
    if n%2==0:
        X = Gamma(int(n/2),1/2,N)
        return np.array(X)
    else:
        x = Gamma(int((n-1))/2,1/2,N)
        y,Z= box_muller(N)
        X = x + Z**2
        return X
    
def t_Student(n,N):
    x,Z = box_muller(N)
    Y = chi_cuadrado(n,N)
    X = Z/np.sqrt(Y/n)
    return X


<font size="6"><center><span style="color:blue">***Práctica 3***</span></center></font>

Obtener mediante simulación el valor de una opción call europea vainilla con dividendos para plazos t=1,2,...,T, pero cuyos incrementos se rigen según una distribución t-Student. Dar la estimación del valor y la precisión y comparar con el resultado de la práctica anterior. Utilizar los mismos datos que en la práctica anterior y n=6. Utilizar para la evolución del subyacente:

$$
S \leftarrow S e^{(r_c - d_c)\Delta t} + \sigma S \sqrt{\Delta t}\frac{t_n}{\sqrt{n/(n-2)}}
$$

<font size="2">**Nota:** Recordar que la varianza de una t-Student con n grados de libertad es $\frac{n}{n-2}$, con lo que al sustituir la Normal por la t-Student, ha de hacerse con ésta tipificada.
    
<font size="2" color=grey>**Nota para comprobación:** $t=1; V_{\mathrm{call}}=1.840173;Varianza=8.169188;Precisión=0.177363$.</font>

In [9]:

r = 0.03
r_c = log(1+r)
K = 14
T = 1
sigma = 0.3
S0 = 14
d_c = 0
d=0
N = 1000
n=6
delta_t = 1/12

def vainilla(r_c,d_c,plazos,delta_t,S0,n,N):
    t = t_Student(n, int(N*(plazos/delta_t)))
    #t = si.t.rvs(df=n, size=int(N*(plazos/delta_t)), random_state=42)
    t = t.reshape(N, int(plazos/delta_t))
    multiplicadores = exp((r_c-d_c)*delta_t) + sigma  * sqrt(delta_t)*(t/(sqrt(n/(n-2))))
    ST= S0*np.prod(multiplicadores, axis=1)
    return ST
plazos = [1,2,3,4]



In [10]:


df = pd.DataFrame([{        "Plazo": i,
        "Valor estimado": calcular_estadisticas_montecarlo(exp(-r_c*T) * np.maximum(vainilla(r_c,d_c, i,delta_t,S0,n,N)-K, 0))[0],
        "Varianza": calcular_estadisticas_montecarlo(exp(-r_c*T) * np.maximum(vainilla(r_c,d_c, i,delta_t,S0,n,N)-K, 0))[1],
        "Precisión (95%)": calcular_estadisticas_montecarlo(exp(-r_c*T) * np.maximum(vainilla(r_c,d_c, i,delta_t,S0,n,N)-K, 0))[2]}for i in plazos ])

imprimir_tabla_estadisticas(df)


      RESULTADOS MONTE CARLO - OPCION CALL EUROPEA VAINILLA
Plazos     |  Valor estimado |        Varianza |    Precisión (95%)
---------------------------------------------------------------------------
1.0        |        1.944092 |        9.438068 |           0.190641
2.0        |        2.941250 |       31.520940 |           0.348397
3.0        |        3.742717 |       60.591378 |           0.483037
4.0        |        4.679940 |       80.472231 |           0.556670

